# Connectivity

## PLI — Phase-Lag Index

In [ ]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

# 1. Setup
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJ = "sub-001"
SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
DATA_DIR = os.path.join(SUBJ_DIR, "data")
EPO_FILE = os.path.join(SUBJ_DIR, "epo_001_raw.fif")

# Load epochs
epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info['sfreq']

# Define bands for connectivity (standard canonical bands)
con_bands = {
    'delta': (1, 4), 
    'theta': (4, 8), 
    'alpha': (8, 12), 
    'beta': (12, 30)
}

# 2. Compute Connectivity (PLI)
results = {}
for band_name, (fmin, fmax) in con_bands.items():
    print(f"Computing {band_name} connectivity...")
    con_obj = spectral_connectivity_epochs(
        epochs,
        method='pli',          # or 'plv' for PLV
        mode='fourier',
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        faverage=True,
        mt_adaptive=False,
        n_jobs=1
    )
    # Extract the actual connectivity matrix
    con_mat = con_obj.get_data(output='dense')  # shape: n_channels x n_channels
    results[band_name] = con_mat


# 3. Save results in subject's data folder
os.makedirs(DATA_DIR, exist_ok=True)

for band_name, con_mat in results.items():
    out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli.npy")
    np.save(out_file, con_mat)
    print(f"Saved {band_name} PLI matrix to {out_file}")


In [ ]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1,150)]

CON_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
}

for SUBJ in SUBJECTS:
    print(f"Processing {SUBJ}...")

    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")

    if not os.path.exists(EPO_FILE):
        print("  Missing epochs file, skipping")
        continue

    os.makedirs(DATA_DIR, exist_ok=True)

    epochs = mne.read_epochs(EPO_FILE, preload=True)
    sfreq = epochs.info["sfreq"]
    n_ch = len(epochs.ch_names)

    for band_name, (fmin, fmax) in CON_BANDS.items():
        print(f"  Computing {band_name} PLV...")

        con = spectral_connectivity_epochs(
            epochs,
            method="pli",
            mode="fourier",
            sfreq=sfreq,
            fmin=fmin,
            fmax=fmax,
            faverage=True,
            n_jobs=1,
            verbose=False
        )

        # Dense square matrix
        mat = con.get_data(output="dense")  # (n_ch, n_ch, 1)
        mat = mat[:, :, 0]

        # HARD ASSERT
        assert mat.shape == (n_ch, n_ch), f"{SUBJ} {band_name} bad shape {mat.shape}"

        out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli_matrix.npy")
        np.save(out_file, mat)

        print(f"    Saved {band_name} PLV: {mat.shape}")


### COMPUTE PLI WINDOWS

In [ ]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 150)]

CON_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
}

# Sliding window parameters
WINDOW_SEC = 2.0  # window length in seconds
STEP_SEC = 1.0    # overlap step

for SUBJ in SUBJECTS:
    print(f"Processing {SUBJ}...")

    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")

    if not os.path.exists(EPO_FILE):
        print("  Missing epochs file, skipping")
        continue

    os.makedirs(DATA_DIR, exist_ok=True)

    epochs = mne.read_epochs(EPO_FILE, preload=True)
    sfreq = epochs.info["sfreq"]
    n_ch = len(epochs.ch_names)

    win_samples = int(WINDOW_SEC * sfreq)
    step_samples = int(STEP_SEC * sfreq)

    for band_name, (fmin, fmax) in CON_BANDS.items():
        print(f"  Computing {band_name} PLI windows...")

        pli_windows = []

        # Loop through each epoch
        for ep_data in epochs.get_data():  # shape: n_ch x n_times
            n_samples = ep_data.shape[1]
            for start in range(0, n_samples - win_samples + 1, step_samples):
                window = ep_data[:, start:start + win_samples][np.newaxis, :, :]
                window_epochs = mne.EpochsArray(window, info=epochs.info)
                con = spectral_connectivity_epochs(
                    window_epochs,
                    method="pli",
                    mode="fourier",
                    sfreq=sfreq,
                    fmin=fmin,
                    fmax=fmax,
                    faverage=True,
                    n_jobs=1,
                    verbose=False
                )
                mat = con.get_data(output="dense")[:, :, 0]  # n_ch x n_ch
                pli_windows.append(mat)

        if len(pli_windows) == 0:
            print(f"    No windows computed for {band_name}, skipping")
            continue

        pli_windows = np.array(pli_windows)  # n_windows x n_ch x n_ch
        out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli_windows.npy")
        np.save(out_file, pli_windows)
        print(f"    Saved {band_name} PLI windows: {pli_windows.shape}")


KeyboardInterrupt: 

In [5]:
import mne
import numpy as np
import os
from mne_connectivity import spectral_connectivity_epochs

# =========================
# CONFIGURATION
# =========================
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 36)]  # overwrite sub-001 to sub-035

CON_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
}

WINDOW_SEC = 2.0   # 2-second windows
STEP_SEC = 1.0     # 1-second overlap

# =========================
# PROCESSING LOOP
# =========================
for SUBJ in SUBJECTS:
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")
    DATA_DIR = os.path.join(SUBJ_DIR, "data")

    if not os.path.exists(EPO_FILE):
        print(f"{SUBJ}: Epochs file missing, skipping")
        continue

    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"Processing {SUBJ}...")

    # --- Load epochs ---
    epochs = mne.read_epochs(EPO_FILE, preload=True, verbose=False)
    sfreq = epochs.info["sfreq"]
    data = epochs.get_data()  # shape: n_epochs x n_channels x n_times
    n_epochs, n_ch, n_times = data.shape

    # --- Sliding window indices ---
    win_samples = int(WINDOW_SEC * sfreq)
    step_samples = int(STEP_SEC * sfreq)
    starts = np.arange(0, n_times - win_samples + 1, step_samples)

    # --- Vectorized window extraction ---
    # Result: (n_epochs * n_windows_per_epoch, n_channels, win_samples)
    windowed_data = np.concatenate([data[:, :, s:s + win_samples] for s in starts], axis=0)
    window_epochs = mne.EpochsArray(windowed_data, info=epochs.info, verbose=False)

    # --- Compute PLI for each band ---
    for band_name, (fmin, fmax) in CON_BANDS.items():
        print(f"  Computing {band_name} PLI windows...")
        con = spectral_connectivity_epochs(
            window_epochs,
            method="pli",
            mode="fourier",
            sfreq=sfreq,
            fmin=fmin,
            fmax=fmax,
            faverage=True,
            n_jobs=-1,
            verbose=False
        )
        # Dense PLI matrix: (n_windows, n_channels, n_channels)
        pli_matrices = con.get_data(output="dense")

        # --- Save ---
        out_file = os.path.join(DATA_DIR, f"{SUBJ}_{band_name}_pli_windows.npy")
        np.save(out_file, pli_matrices)
        print(f"    Saved {pli_matrices.shape[0]} windows for {band_name}")

print("\nDone! Sliding window PLI saved for sub-001 to sub-035.")


Processing sub-001...
  Computing delta PLI windows...


C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:38: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:56: RuntimeWarning: There were no Annotations stored in <EpochsArray | 231 events (all good), 0 – 1.996 s (baseline off), ~52.9 MiB, data loaded,
 '1': 231>, so metadata was not modified.
  con = spectral_connectivity_epochs(
C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:56: RuntimeWarning: fmin=1.000 Hz corresponds to 2.000 < 5 cycles based on the epoch length 2.000 sec, need at least 5.000 sec epochs or fmin=2.500. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    Saved 60 windows for delta
  Computing theta PLI windows...


C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:56: RuntimeWarning: There were no Annotations stored in <EpochsArray | 231 events (all good), 0 – 1.996 s (baseline off), ~52.9 MiB, data loaded,
 '1': 231>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved 60 windows for theta
  Computing alpha PLI windows...


...

    Saved 60 windows for theta
  Computing alpha PLI windows...


C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:56: RuntimeWarning: There were no Annotations stored in <EpochsArray | 300 events (all good), 0 – 1.996 s (baseline off), ~68.7 MiB, data loaded,
 '1': 300>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved 60 windows for alpha
  Computing beta PLI windows...


C:\Users\User\AppData\Local\Temp\ipykernel_27620\3664300381.py:56: RuntimeWarning: There were no Annotations stored in <EpochsArray | 300 events (all good), 0 – 1.996 s (baseline off), ~68.7 MiB, data loaded,
 '1': 300>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    Saved 60 windows for beta

Done! Sliding window PLI saved for sub-001 to sub-035.
